# Data Cleaning (Mourad)
## Credit Card Customers Churn

Author: [Mourad Askar](maskar2@depaul.edu)  
Created: _7-FEB-2021_

# The Dataset

**Source**  
The dataset was downloaded from kaggle:  
https://www.kaggle.com/sakshigoyal7/credit-card-customers


**Dataset statistics:**    
Number of observations	10,127  
Number of variables		20  (After dropping CLIENTNUM index)

Missing cells		0  
Missing cells (%)	0.0%  

Duplicate rows	0  
Duplicate rows (%)	0.0%  

**Variable types**  
Numeric:	15  
Categorical:	6  

**Target Variable**  
Attrition_Flag

Attritted customers proportion is 16%


|Variable|Type|Description|
|-|-|-|
|Clientnum|	Num|	Client number. Unique identifier for the customer holding the account|
|Attrition_Flag|	Char|	Internal event (customer activity) variable - if the account is closed then 1 else 0|
|Customer_Age|	Num|	Demographic variable - Customer's Age in Years|
|Gender|	Char|	Demographic variable - M=Male, F=Female|
|Dependent_count|	Num|	Demographic variable - Number of dependents|
|Education_Level|	Char|	Demographic variable - Educational Qualification of the account holder (example: high school, college graduate, etc.)|
|Marital_Status|	Char|	Demographic variable - Married, Single, Unknown|
|Income_Category|	Char|	Demographic variable - Annual Income Category of the account holder (< $40K, $40K - 60K, $60K - $80K, $80K-$120K, > $120K, Unknown)|
|Card_Category|	Char|	Product Variable - Type of Card (Blue, Silver, Gold, Platinum)|
|Months_on_book|	Num|	Months on book (Time of Relationship)|
|Total_Relationship_Count|	Num|	Total no. of products held by the customer|
|Months_Inactive_12_mon|	Num|	No. of months inactive in the last 12 months|
|Contacts_Count_12_mon|	Num|	No. of Contacts in the last 12 months|
|Credit_Limit|	Num|	Credit Limit on the Credit Card|
|Total_Revolving_Bal|	Num|	Total Revolving Balance on the Credit Card|
|Avg_Open_To_Buy|	Num|	Open to Buy Credit Line (Average of last 12 months)|
|Total_Amt_Chng_Q4_Q1|	Num|	Change in Transaction Amount (Q4 over Q1)| 
|Total_Trans_Amt|	Num|	Total Transaction Amount (Last 12 months)|
|Total_Trans_Ct|	Num|	Total Transaction Count (Last 12 months)|
|Total_Ct_Chng_Q4_Q1|	Num|	Change in Transaction Count (Q4 over Q1)|
|Avg_Utilization_Ratio|	Num|	Average Card Utilization Ratio|


In [1]:
import pandas as pd
import numpy as np
import klib
import pandas_profiling as pp
import sweetviz
import sklearn
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from platform import python_version

pd.set_option('max_columns',200)
pd.set_option('display.precision',2)

print('python',python_version())
print(np.__name__, np.__version__)
print(pd.__name__, pd.__version__)
print(klib.__name__, klib.__version__)
#print(pp.__name__, pp.__version__)
print(sklearn.__name__, sklearn.__version__)
print(sweetviz.__name__, sweetviz.__version__)

python 3.9.2
numpy 1.20.1
pandas 1.1.4
klib 0.1.5
sklearn 0.24.1
sweetviz 2.0.9


The source of the daset

## Read data and understand structure

In [2]:
# Read data from source CSV and drop the last 2 irrelevant columns
df = pd.read_csv('./data/BankChurners.csv', index_col=0).iloc[:, :-2]

# Reset index IDs and drop old index values
df.reset_index(drop=True, inplace=True)

print(df.info())
display(df.sample(5).T)  # Transpose just for easier viewing

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Attrition_Flag            10127 non-null  object 
 1   Customer_Age              10127 non-null  int64  
 2   Gender                    10127 non-null  object 
 3   Dependent_count           10127 non-null  int64  
 4   Education_Level           10127 non-null  object 
 5   Marital_Status            10127 non-null  object 
 6   Income_Category           10127 non-null  object 
 7   Card_Category             10127 non-null  object 
 8   Months_on_book            10127 non-null  int64  
 9   Total_Relationship_Count  10127 non-null  int64  
 10  Months_Inactive_12_mon    10127 non-null  int64  
 11  Contacts_Count_12_mon     10127 non-null  int64  
 12  Credit_Limit              10127 non-null  float64
 13  Total_Revolving_Bal       10127 non-null  int64  
 14  Avg_Op

,2038,4139,1675,719,3016
Attrition_Flag,Existing Customer,Existing Customer,Existing Customer,Existing Customer,Attrited Customer
Customer_Age,52,50,56,39,51
Gender,F,F,M,M,F
Dependent_count,3,3,1,2,1
Education_Level,Doctorate,Doctorate,College,Uneducated,Graduate
Marital_Status,Single,Single,Married,Single,Single
Income_Category,Unknown,Less than $40K,$80K - $120K,$40K - $60K,Less than $40K
Card_Category,Blue,Blue,Blue,Blue,Blue
Months_on_book,36,39,45,32,37
Total_Relationship_Count,3,6,5,6,1


## Clean data and feature names

In [3]:
klib.missingval_plot(df)

No missing values found in the dataset.


In [4]:
df_c = klib.data_cleaning(df, convert_dtypes=False)
print(df_c.info())

Shape of cleaned data: (10127, 20)Remaining NAs: 0

Changes:
Dropped rows: 0
     of which 0 duplicates. (Rows: [])
Dropped columns: 0
     of which 0 single valued.     Columns: []
Dropped missing values: 0
Reduced memory by at least: -0.39 MB (--25.16%)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   attrition_flag            10127 non-null  object 
 1   customer_age              10127 non-null  int64  
 2   gender                    10127 non-null  object 
 3   dependent_count           10127 non-null  int64  
 4   education_level           10127 non-null  object 
 5   marital_status            10127 non-null  object 
 6   income_category           10127 non-null  object 
 7   card_category             10127 non-null  object 
 8   months_on_book            10127 non-null  int64  
 9   total_relationship_count

# Generate Exploratory Data Analysis Report

> I comment out most of this section as not to re-generate the reports everytime  
> Output can be found at **"01_DSC478_EDA_PP.html"** and **"01_DSC478_EDA_sweetviz.html"**

#### Pandas Profiling Report
> I use the suffix "_new" in the file name to prevent over-writing existing report

In [5]:
#pp_report = pp.ProfileReport(df_c, explorative=True, lazy=False)
#pp_report.to_file('01_DSC478_EDA_PP_new.html')

#### SweetViz Report
> I use the suffix "_new" in the file name to prevent over-writing existing report

In [6]:
#a = pd.get_dummies(df_c, columns=['attrition_flag'])
#b = klib.data_cleaning(a).drop(columns='attrition_flag_existing_customer')
#sv = sweetviz.analyze(b, target_feat='attrition_flag_attrited_customer')
#sv.show_html('01_DSC478_EDA_sweetviz_new.html', layout='vertical', open_browser=False)
#sv.show_notebook()

# Datasets (1)

# Fix the columns order 

In [7]:
# Decide the new columns order
new_col_order = ['customer_age', 'dependent_count', 'months_on_book',
                'total_relationship_count', 'months_inactive_12_mon',
                'contacts_count_12_mon', 'credit_limit', 'total_revolving_bal',
                'avg_open_to_buy', 'total_amt_chng_q4_q1', 'total_trans_amt',
                'total_trans_ct', 'total_ct_chng_q4_q1', 'avg_utilization_ratio',
                'gender','education_level','marital_status','income_category',
                'card_category','attrition_flag'
               ]
# Apply the new columns order
df_c2 = df_c.reindex(columns=new_col_order)
# Replace the values in attrition_flag with binary values, 1 for attrited_customer
df_c2['attrition_flag'].replace({'Existing Customer':False, 'Attrited Customer':True}, inplace=True)
# Rename column attrition_flag to attrited_customer
df_c2.rename(columns={'attrition_flag':'attrited_customer'}, inplace=True)
df_c2.head().T

,0,1,2,3,4
customer_age,45,49,51,40,40
dependent_count,3,5,3,4,3
months_on_book,39,44,36,34,21
total_relationship_count,5,6,4,3,5
months_inactive_12_mon,1,1,1,4,1
contacts_count_12_mon,3,2,0,1,0
credit_limit,1.3e+04,8.3e+03,3.4e+03,3.3e+03,4.7e+03
total_revolving_bal,777,864,0,2517,0
avg_open_to_buy,1.2e+04,7.4e+03,3.4e+03,8e+02,4.7e+03
total_amt_chng_q4_q1,1.3,1.5,2.6,1.4,2.2


# Write out clean file ready for next steps

In [8]:
# Export cleaned data to CSV
df_c2.to_csv('./data/bank_churners_clean.csv', index_label='index')

In [9]:
# Read clean data from CSV (To test exported data and as a reference code)
df_csv = pd.read_csv('./data/bank_churners_clean.csv', index_col=0)
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_age              10127 non-null  int64  
 1   dependent_count           10127 non-null  int64  
 2   months_on_book            10127 non-null  int64  
 3   total_relationship_count  10127 non-null  int64  
 4   months_inactive_12_mon    10127 non-null  int64  
 5   contacts_count_12_mon     10127 non-null  int64  
 6   credit_limit              10127 non-null  float64
 7   total_revolving_bal       10127 non-null  int64  
 8   avg_open_to_buy           10127 non-null  float64
 9   total_amt_chng_q4_q1      10127 non-null  float64
 10  total_trans_amt           10127 non-null  int64  
 11  total_trans_ct            10127 non-null  int64  
 12  total_ct_chng_q4_q1       10127 non-null  float64
 13  avg_utilization_ratio     10127 non-null  float64
 14  gender

# Prepare Train / Test Datasets

In [10]:
# Check dimensions of source data
df_csv.shape

(10127, 20)

In [11]:
# Split data in to Features X and Target y
X = df_csv.iloc[:,:-1]
y = df_csv.iloc[:,-1]
print('X and y shapes:')
print(X.shape,y.shape,'\n')
print('Target Ratio:')
print(y.value_counts(normalize=True, dropna=False),'\n')

X and y shapes:
(10127, 19) (10127,) 

Target Ratio:
False    0.84
True     0.16
Name: attrited_customer, dtype: float64 



In [12]:
# Split data into Train and Test
# We stratify to make sure target representation is kept in the new datasets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=.2,
                                                    stratify=y,
                                                    random_state=2021)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8101, 19), (2026, 19), (8101,), (2026,))

In [13]:
# Merge split Features and Target to export into single CSV for each of Train and Test
X_train.merge(y_train, left_index=True, right_index=True)\
                                .to_csv('./data/bank_churners_train.csv', index_label='index')
X_test.merge(y_test, left_index=True, right_index=True)\
                                .to_csv('./data/bank_churners_test.csv', index_label='index')

# NOTE: I keep the original index order in the split Train / Test CSV
#       for tracking back to source purposes

> **NOTE:**
> * The **Test dataset** is to be kept away until the last step for evaluating the Final Model.
> * Both **Training and Validation** should be done on the **Train dataset**

In [14]:
# Read [Train] data from CSV (To test exported data and as a reference code)
df_train = pd.read_csv('./data/bank_churners_train.csv', index_col=0)
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:,-1]
print('X and y shapes:')
print(X_train.shape,y_train.shape,'\n')
print('Target Ratio:')
print(y_train.value_counts(normalize=True, dropna=False),'\n')
df_train.info()

X and y shapes:
(8101, 19) (8101,) 

Target Ratio:
False    0.84
True     0.16
Name: attrited_customer, dtype: float64 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8101 entries, 3066 to 1919
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_age              8101 non-null   int64  
 1   dependent_count           8101 non-null   int64  
 2   months_on_book            8101 non-null   int64  
 3   total_relationship_count  8101 non-null   int64  
 4   months_inactive_12_mon    8101 non-null   int64  
 5   contacts_count_12_mon     8101 non-null   int64  
 6   credit_limit              8101 non-null   float64
 7   total_revolving_bal       8101 non-null   int64  
 8   avg_open_to_buy           8101 non-null   float64
 9   total_amt_chng_q4_q1      8101 non-null   float64
 10  total_trans_amt           8101 non-null   int64  
 11  total_trans_ct            8101 non-null   int64  

In [15]:
# Read [Test] data from CSV (To test exported data and as a reference code)
df_test = pd.read_csv('./data/bank_churners_test.csv', index_col=0)
X_test = df_test.iloc[:,:-1]
y_test = df_test.iloc[:,-1]
print('X and y shapes:')
print(X_test.shape,y_test.shape,'\n')
print('Target Ratio:')
print(y_test.value_counts(normalize=True, dropna=False),'\n')
df_test.info()

X and y shapes:
(2026, 19) (2026,) 

Target Ratio:
False    0.84
True     0.16
Name: attrited_customer, dtype: float64 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2026 entries, 9987 to 4830
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_age              2026 non-null   int64  
 1   dependent_count           2026 non-null   int64  
 2   months_on_book            2026 non-null   int64  
 3   total_relationship_count  2026 non-null   int64  
 4   months_inactive_12_mon    2026 non-null   int64  
 5   contacts_count_12_mon     2026 non-null   int64  
 6   credit_limit              2026 non-null   float64
 7   total_revolving_bal       2026 non-null   int64  
 8   avg_open_to_buy           2026 non-null   float64
 9   total_amt_chng_q4_q1      2026 non-null   float64
 10  total_trans_amt           2026 non-null   int64  
 11  total_trans_ct            2026 non-null   int64  

# Datasets (2)

# Create dummies for categorical features

In [16]:
df_dummies = pd.get_dummies(df_c,
                            columns=['gender','education_level','marital_status',
                                     'income_category','card_category','attrition_flag'],
                            prefix=['gender','ed_lvl','marital','income','card','']
                           ).drop(columns='_Existing Customer')
df_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10127 entries, 0 to 10126
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_age              10127 non-null  int64  
 1   dependent_count           10127 non-null  int64  
 2   months_on_book            10127 non-null  int64  
 3   total_relationship_count  10127 non-null  int64  
 4   months_inactive_12_mon    10127 non-null  int64  
 5   contacts_count_12_mon     10127 non-null  int64  
 6   credit_limit              10127 non-null  float64
 7   total_revolving_bal       10127 non-null  int64  
 8   avg_open_to_buy           10127 non-null  float64
 9   total_amt_chng_q4_q1      10127 non-null  float64
 10  total_trans_amt           10127 non-null  int64  
 11  total_trans_ct            10127 non-null  int64  
 12  total_ct_chng_q4_q1       10127 non-null  float64
 13  avg_utilization_ratio     10127 non-null  float64
 14  gender

# Clean feature names after introducing new dummy features

In [17]:
rm_dollar = lambda x: str(x).replace('dollar_','')

df_dummies_c = klib.data_cleaning(df_dummies).rename(columns=rm_dollar)
df_dummies_c.info()

Long column names detected (>25 characters). Consider renaming the following columns ['income_dollar_40k_dollar_60k', 'income_dollar_60k_dollar_80k', 'income_dollar_80k_dollar_120k', 'income_less_than_dollar_40k'].
Shape of cleaned data: (10127, 38)Remaining NAs: 0

Changes:
Dropped rows: 0
     of which 0 duplicates. (Rows: [])
Dropped columns: 0
     of which 0 single valued.     Columns: []
Dropped missing values: 0
Reduced memory by at least: 0.77 MB (-45.29%)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10127 entries, 0 to 10126
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_age              10127 non-null  int8   
 1   dependent_count           10127 non-null  int8   
 2   months_on_book            10127 non-null  int8   
 3   total_relationship_count  10127 non-null  int8   
 4   months_inactive_12_mon    10127 non-null  int8   
 5   contacts_count_12_mon     10127

# Write out clean file ready for next steps (with Dummies)

In [18]:
# Export cleaned data to CSV
df_dummies_c.to_csv('./data/bank_churners_clean_dummies.csv', index_label='index')

In [19]:
# Read clean data from CSV (To test exported data and as a reference code)
df_dummies_csv = pd.read_csv('./data/bank_churners_clean_dummies.csv', index_col=0)
df_dummies_csv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10127 entries, 0 to 10126
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_age              10127 non-null  int64  
 1   dependent_count           10127 non-null  int64  
 2   months_on_book            10127 non-null  int64  
 3   total_relationship_count  10127 non-null  int64  
 4   months_inactive_12_mon    10127 non-null  int64  
 5   contacts_count_12_mon     10127 non-null  int64  
 6   credit_limit              10127 non-null  float64
 7   total_revolving_bal       10127 non-null  int64  
 8   avg_open_to_buy           10127 non-null  float64
 9   total_amt_chng_q4_q1      10127 non-null  float64
 10  total_trans_amt           10127 non-null  int64  
 11  total_trans_ct            10127 non-null  int64  
 12  total_ct_chng_q4_q1       10127 non-null  float64
 13  avg_utilization_ratio     10127 non-null  float64
 14  gender

# Prepare Train / Test Datasets

In [20]:
# Check dimensions of source data
df_dummies_csv.shape

(10127, 38)

In [21]:
# Split data in to Features X and Target y
X = df_dummies_csv.iloc[:,:-1]
y = df_dummies_csv.iloc[:,-1]
print('X and y shapes:')
print(X.shape,y.shape,'\n')
print('Target Ratio:')
print(y.value_counts(normalize=True, dropna=False),'\n')

X and y shapes:
(10127, 37) (10127,) 

Target Ratio:
0    0.84
1    0.16
Name: attrited_customer, dtype: float64 



In [22]:
# Split data into Train and Test
# We stratify to make sure target representation is kept in the new datasets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=.2,
                                                    stratify=y,
                                                    random_state=2021)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8101, 37), (2026, 37), (8101,), (2026,))

In [23]:
# Merge split Features and Target to export into single CSV for each of Train and Test
X_train.merge(y_train, left_index=True, right_index=True, validate='1:1')\
                                .to_csv('./data/bank_churners_train_dummies.csv', index_label='index')
X_test.merge(y_test, left_index=True, right_index=True, validate='1:1')\
                                .to_csv('./data/bank_churners_test_dummies.csv', index_label='index')

# NOTE: I keep the original index order in the split Train / Test CSV
#       for tracking back to source purposes

> **NOTE:**
> * The **Test dataset** is to be kept away until the last step for evaluating the Final Model.
> * Both **Training and Validation** should be done on the **Train dataset**

In [24]:
# Read [Train] data from CSV (To test exported data and as a reference code)
df_train = pd.read_csv('./data/bank_churners_train_dummies.csv', index_col=0)
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:,-1]
print('X and y shapes:')
print(X_train.shape,y_train.shape,'\n')
print('Target Ratio:')
print(y_train.value_counts(normalize=True, dropna=False),'\n')
df_train.info()

X and y shapes:
(8101, 37) (8101,) 

Target Ratio:
0    0.84
1    0.16
Name: attrited_customer, dtype: float64 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8101 entries, 3066 to 1919
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_age              8101 non-null   int64  
 1   dependent_count           8101 non-null   int64  
 2   months_on_book            8101 non-null   int64  
 3   total_relationship_count  8101 non-null   int64  
 4   months_inactive_12_mon    8101 non-null   int64  
 5   contacts_count_12_mon     8101 non-null   int64  
 6   credit_limit              8101 non-null   float64
 7   total_revolving_bal       8101 non-null   int64  
 8   avg_open_to_buy           8101 non-null   float64
 9   total_amt_chng_q4_q1      8101 non-null   float64
 10  total_trans_amt           8101 non-null   int64  
 11  total_trans_ct            8101 non-null   int64  
 12  to

In [25]:
# Read [Test] data from CSV (To test exported data and as a reference code)
df_test = pd.read_csv('./data/bank_churners_test_dummies.csv', index_col=0)
X_test = df_test.iloc[:,:-1]
y_test = df_test.iloc[:,-1]
print('X and y shapes:')
print(X_test.shape,y_test.shape,'\n')
print('Target Ratio:')
print(y_test.value_counts(normalize=True, dropna=False),'\n')
df_test.info()

X and y shapes:
(2026, 37) (2026,) 

Target Ratio:
0    0.84
1    0.16
Name: attrited_customer, dtype: float64 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2026 entries, 9987 to 4830
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_age              2026 non-null   int64  
 1   dependent_count           2026 non-null   int64  
 2   months_on_book            2026 non-null   int64  
 3   total_relationship_count  2026 non-null   int64  
 4   months_inactive_12_mon    2026 non-null   int64  
 5   contacts_count_12_mon     2026 non-null   int64  
 6   credit_limit              2026 non-null   float64
 7   total_revolving_bal       2026 non-null   int64  
 8   avg_open_to_buy           2026 non-null   float64
 9   total_amt_chng_q4_q1      2026 non-null   float64
 10  total_trans_amt           2026 non-null   int64  
 11  total_trans_ct            2026 non-null   int64  
 12  to

# END